In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn import metrics


In [2]:
data=pd.read_csv("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
data.head(3)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."


In [3]:
sentiment_data = data[['reviews.rating', 'reviews.text', 'reviews.title']]
sentiment_data.head()

,reviews.rating,reviews.text,reviews.title
0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...
1,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...
2,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...
3,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...
4,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...


In [4]:
## prepare data
def sentiment_labeler(score):
    if (score==5) or (score==4):
        return "Positive"
    elif (score==3):
        return "Neutral"
    else:
        return "Negative"
    
sentiment_data["sentiment_label"]=sentiment_data["reviews.rating"].apply(sentiment_labeler)
sentiment_data.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,reviews.rating,reviews.text,reviews.title,sentiment_label
0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Neutral
1,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,Positive
2,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,Positive
3,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Positive
4,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Positive


### prepare train, test data


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split_data=StratifiedShuffleSplit(n_splits=5, test_size=0.2)
for tr_indx, ts_indx in split_data.split(sentiment_data, sentiment_data["reviews.rating"]):
    train_data=sentiment_data.reindex(tr_indx)
    test_data=sentiment_data.reindex(ts_indx)

In [6]:
## set up stanfordcorenlp

In [7]:
from pycorenlp import StanfordCoreNLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import numpy as np
import string

In [11]:
## A class to parse chunk data
import nltk
class Chunker():
	def __init__(self, n):
		# Load the corpus to train the chunk tagger
		conll = nltk.corpus.conll2000.chunked_sents()
		treebank = nltk.corpus.treebank_chunk.chunked_sents()
		data = conll + treebank

		chunks = [ nltk.chunk.tree2conlltags(tree) for tree in data ]
		chunks = [ [(tag[1], tag[2]) for tag in tags] for tags in chunks ]

		train_chunks = chunks

		# Train the chunk tagger
		self.chunk_tagger = None
		if n == "Unigram":
			self.chunk_tagger = nltk.tag.UnigramTagger(train_chunks)
		elif n == "Bigram":
			self.chunk_tagger = nltk.tag.BigramTagger(train_chunks)
		elif n == "Trigram":
			self.chunk_tagger = nltk.tag.TrigramTagger(train_chunks)
		else:
			chunker = nltk.tag.UnigramTagger(train_chunks)
			chunker = nltk.tag.BigramTagger(train_chunks, backoff=chunker)
			chunker = nltk.tag.TrigramTagger(train_chunks, backoff=chunker)
			self.chunk_tagger = chunker

	def parseTree(self, tokens):
		# Gather the words, tags, and chunks
		words = [w for (w,t) in tokens]
		tags = [t for (w,t) in tokens]
		chunks = self.chunk_tagger.tag(tags)

		# Sanity check
		assert len(words) == len(tags)
		assert len(words) == len(chunks)

		# Build the parse tree
		l = []
		for i in range(0,len(words)):
			if chunks[i][1]:
				l.append( ' '.join([words[i], tags[i], chunks[i][1]]) )

		return nltk.chunk.conllstr2tree('\n'.join(l))

def chunkLabels(tree):
	l = []
	for t in tree.subtrees():
		if t.label() != 'S':
			l.append(t.label())
	return ' '.join(l)

ch_uni = Chunker("Unigram") # Train the chunk tagger
ch_bi = Chunker("Bigram") # Train the chunk tagger
ch_tri = Chunker("Trigram") # Train the chunk tagger

# Helper function
def GetChunkVectorsTFIDF(X):
    chunks=X.apply(str.split) # Tokenize
    chunks=chunks.apply(nltk.pos_tag) # POS tag
    chunks=chunks.apply(ch_uni.parseTree) # Generate parse tree
    chunks=chunks.apply(chunkLabels) # Extract chunk labels
    chunks=CountVectorizer().fit_transform(chunks)
    chunks=TfidfTransformer().fit_transform(chunks)
    return chunks

def GetChunkVectorsCount(X):
    chunks=X.apply(str.split) # Tokenize
    chunks=chunks.apply(nltk.pos_tag) # POS tag
    chunks=chunks.apply(ch_uni.parseTree) # Generate parse tree
    chunks=chunks.apply(chunkLabels) # Extract chunk labels
    chunks=CountVectorizer().fit_transform(chunks)
    return chunks


In [8]:
## helper function to extract pos tag features

def extract_POS(statements):
    print('Extracting POS Tags')
    pos_tags = POS_tagging(statements,return_word_tag_pairs=False)
    bigrams_pos = POS_groupping(pos_tags, grams=2)
    trigrams_pos =POS_groupping(pos_tags, grams=3)
    print('Finished')
    return pos_tags,bigrams_pos,trigrams_pos

##
def POS_tagging(statements, return_word_tag_pairs = False):
    core_nlp = StanfordCoreNLP('http://localhost:9000')
    print("NLP_Task ready to use.")
    POS_tags = list()
    for statement in statements:
        statement_tags = list()
        annotations = core_nlp.annotate(statement, properties={
            'annotators': 'tokenize,pos',
            'outputFormat': 'json'
            })
        for output in annotations['sentences']:
            statement_tags.append('<s>')
            previous = ''
            for token in output['tokens']:
                if return_word_tag_pairs:
                    statement_tags.append(token['word']+'/'+token['pos'])
                else:
                    statement_tags.append(token['pos'])

        POS_tags.append(statement_tags)
    return POS_tags

## 
def POS_groupping(sentences_pos,grams=1):
    result = list()
    for sentence_tags in sentences_pos:
        tag_group = list()
        for index, each_tag in enumerate(sentence_tags):
            if index < len(sentence_tags)-grams and len(sentence_tags)>=grams:
                format_str = str()
                for i in range(0,grams):
                    format_str += sentence_tags[index+i]
                    if i<grams-1:
                        format_str += '_'
                tag_group.append(format_str)
        result.append(tag_group)
    return result

##
def RemoveConsecutiveTags(list_to_remove, postags,ignore_punctuation=False):
    withoutConsecutiveTags = list()
    for each_tag in postags:
        removed = list()
        previous = ''
        for tt in each_tag:
            if tt != previous:
                if not ignore_punctuation:
                    removed.append(tt)
                elif tt not in string.punctuation:
                    removed.append(tt)
                previous = tt
            elif tt not in list_to_remove:
                removed.append(tt)
                previous = tt
        withoutConsecutiveTags.append(removed)
    return withoutConsecutiveTags

In [9]:
## extracting pos-tag features
unigram_pos_tr, bigrams_pos_tr, trigram_pos_tr = extract_POS(statements=train_data['reviews.text'])
unigram_pos_ts, bigrams_pos_ts, trigram_pos_ts = extract_POS(statements=test_data['reviews.text'])

Extracting POS Tags
NLP_Task ready to use.


Exception: Check whether you have started the CoreNLP server e.g.
$ cd stanford-corenlp-full-2015-12-09/ 
$ java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer

In [ ]:
## remove duplicated pos tag

# For review data
list_to_remove = ['NNP','CD']

removed_pos_tr =RemoveConsecutiveTags(list_to_remove,unigram_pos)
removed_pos_bigrams_tr = POS_groupping(grams=2,sentences_pos=removed_pos_tr)
removed_pos_trigrams_tr = POS_groupping(grams=3,sentences_pos=removed_pos_tr)

removed_pos_ts =RemoveConsecutiveTags(list_to_remove,unigram_pos)
removed_pos_bigrams_ts = POS_groupping(grams=2,sentences_pos=removed_pos_ts)
removed_pos_trigrams_ts = POS_groupping(grams=3,sentences_pos=removed_pos_ts)

In [ ]:
sentiment_feats= pd.DataFrame()
sentiment_feats['reviews'] = sentiment_data['reviews.text']

list_to_remove = ['NNP','CD']
sentiment_feats['pos_unig_tr'] = [" ".join(x).replace('<s>','').replace('$','dollar').strip() for x in removed_pos_tr]
sentiment_feats['pos_big_tr'] = [" ".join(x).replace('$','dollar').strip() for x in removed_pos_bigrams_tr]
sentiment_feats['pos_trig_tr'] = [" ".join(x).replace('$','dollar').strip() for x in removed_pos_trigrams_tr]
sentiment_feats.head()

In [ ]:
###
def GetFeaturesFromPOS(training_data, user_defined_vocabulary=None):
    user_defined_vocabulary = [x.lower().replace('$','dollar') for x in user_defined_vocabulary]

    # making string of the data
    training_str = [" ".join(x) for x in training_data]

    #replace $ by dollar
    training_str = [x.replace('$', 'dollar').replace('<s>','sos') for x in training_str]

    # features using binary iformation
    oneHotVectorizer = CountVectorizer(vocabulary=user_defined_vocabulary,binary=True)
    tr_onehot = oneHotVectorizer.fit_transform(training_str).toarray()
    print(oneHotVectorizer.vocabulary_)

    # features using no-binary information (counting)
    countVectorizer = CountVectorizer(vocabulary=user_defined_vocabulary,binary=True)
    tr_count = countVectorizer.fit_transform(training_str).toarray()

    # features using tf-idf vectors
    tfIdfVectorizer = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)
    tr_tfidf = tfIdfVectorizer.fit_transform(tr_count)

    return tr_onehot, tr_count, tr_tfidf

In [ ]:
pos_relevant_unigrams =  ['VBZ', 'DT', 'NNPS', 'VBP', 'JJ', 'IN', 'WRB', 'VBD', 'PRP', 'RP', 'WDT', 'VB', 'NNP', 'VBG', 'PRP$', 'VBN', 'CD', 'RB', 'WP', 'JJS', 'JJR', 'EX', 'RBS', 'FW', 'LS'] 
amazonRev_onehot_unigram_tr, amazonRev_count_unigram_tr, amazonRev_tfidf_unigram_tr = GetFeaturesFromPOS(training_data=removed_pos_tr, user_defined_vocabulary=pos_relevant_unigrams)

pos_relevant_bigrams = ['NNPS_VBP', 'VB_NNP', 'IN_DT', 'VB_JJ', 'JJ_CD', 'CD_NNS', 'DT_JJS', 'JJR_IN', 'IN_CD', 'CC_IN', 'RB_VBD', 'CD_NN', 'NN_TO', 'JJR_JJ', 'VB_CD'] 
amazonRev_onehot_bigram_tr, amazonRev_count_bigram_tr, amazonRev_tfidf_bigram_tr = GetFeaturesFromPOS(training_data=removed_pos_bigrams_tr, user_defined_vocabulary=pos_relevant_bigrams)

pos_relevant_trigrams = ['VBD_VBN_IN', 'IN_DT_JJ', 'CD_NN_IN', 'IN_CD_NNS', 'IN_DT_NN', 'DT_JJ_CD', 'MD_VB_IN', 'JJS_JJ_NN', 'CC_JJ_NNS', 'JJ_NNS_VBP', 'VBP_CD_NN', 'sos_JJR_IN', 'IN_DT_NNS','JJ_NN_MD']
amazonRev_onehot_trigram_tr, amazonRev_count_trigram_tr, amazonRev_tfidf_trigram_tr = GetFeaturesFromPOS(training_data=removed_pos_trigrams_tr, user_defined_vocabulary=pos_relevant_trigrams)

In [ ]:
sum(amazonRev_onehot_unigram)

In [ ]:
sum(amazonRev_onehot_bigram)

In [ ]:
sum(amazonRev_onehot_trigram)

In [ ]:
sentiment_feats['pos_unigrams_1hot_tr'] =  [str(x) for x in amazonRev_onehot_unigram_tr]
sentiment_feats['pos_bigrams_1hot_tr'] = [str(x) for x in amazonRev_onehot_bigram_tr]
sentiment_feats['pos_trigrams_1hot_tr'] = [str(x) for x in amazonRev_onehot_trigram_tr]

sentiment_feats['pos_unigrams_count_tr'] =  [str(x) for x in amazonRev_count_unigram_tr]
sentiment_feats['pos_bigrams_count_tr'] = [str(x) for x in amazonRev_count_bigram_tr]
sentiment_feats['pos_trigrams_count_tr'] = [str(x) for x in amazonRev_count_trigram_tr]

sentiment_feats['pos_unigrams_tfidf_tr'] =  [str(x) for x in amazonRev_tfidf_unigram_tr]
sentiment_feats['pos_bigrams_tfidf_tr'] = [str(x) for x in amazonRev_tfidf_bigram_tr]
sentiment_feats['pos_trigrams_tfidf_tr'] = [str(x) for x in amazonRev_tfidf_trigram_tr]

sentiment_feats.head()

In [ ]:

sentiment_feats['chunks_tfidf_tr'] = GetChunkVectorsTFIDF(train_data['reviews.text'])
sentiment_feats['chunks_count_tr'] = GetChunkVectorsCount(train_data['reviews.text'])

sentiment_feats['chunks_tfidf_ts'] = GetChunkVectorsTFIDF(test_data['reviews.text'])
sentiment_feats['chunks_count_ts'] = GetChunkVectorsCount(test_data['reviews.text'])

In [ ]:
### train ML classifier

nb = MultinomialNB()
nb.fit(sentiment_feats[['pos_unigrams_1hot_tr', 'pos_unigrams_count_tr', 'pos_unigrams_tfidf_tr', 'chunks_tfidf_tr']])
#predicted_nb = nb.predict( )

In [ ]:
## use different features

unigram_pos_str = [" ".join(x) for x in unigram_pos]
bigram_pos_str = [" ".join(x) for x in bigrams_pos]
trigram_pos_str = [" ".join(x) for x in trigram_pos]

cv_uni = CountVectorizer()
pos_uni_feats = cv_uni.fit_transform(unigram_pos_str).toarray()

## Sentiment Analysis with TextBlob

In [ ]:
from textblob import TextBlob
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer
from textblob.np_extractors import ConllExtractor

import nltk
nltk.download('movie_reviews')
nltk.download('conll2000')

In [ ]:
def extractTextBlobFeatures(corpus):
    extractor = ConllExtractor()
    text_blob_features = np.zeros((len(corpus),4))
    blob_sentiment_analyzer = Blobber(analyzer=NaiveBayesAnalyzer())
    for i,each_text in enumerate(corpus):
        text_blob_features[i,0]=blob_sentiment_analyzer(each_text).sentiment[1]
        text_blob_features[i,1]=blob_sentiment_analyzer(each_text).sentiment[2]
        text_blob_features[i,2]= TextBlob(each_text).subjectivity
        noun_phrase_extractor = TextBlob(each_text, np_extractor=extractor)
        text_blob_features[i,3]= len(noun_phrase_extractor.noun_phrases)
    return text_blob_features

In [ ]:
X_train=train_data['reviews.text']
y_train=train_data["sentiment_label"]
X_test=test_data["reviews.text"]
y_test=test_data["sentiment_label"]

In [ ]:
Xtr_tb_features = extractTextBlobFeatures(X_train)
Xte_tb_features = extractTextBlobFeatures(X_test)
ytr_tb_features = extractTextBlobFeatures(y_train)
yte_tb_features = extractTextBlobFeatures(y_test)

In [ ]:
Xtr_tb_features

In [ ]:
# Xtr= np.array(Xtr_tb_features.ravel(),)
# ytr=np.array(ytr_tb_features.ravel(),)

# Xte = np.array(Xte_tb_features.ravel(),)
# yte = np.array(yte_tb_features.ravel(),)

In [ ]:
# ## use multinomial NB classifier
# clf_multiNB_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_nominalNB", MultinomialNB())])
# clf_multiNB_pipe.fit(Xtr, ytr)
# predicted_nb = clf_multiNB_pipe.predict(Xte)
# print("accuracy metrics for training naive bayes classifier:\n",metrics.classification_report(yte, predicted_nb, target_names = ['1','0','-1']))
# print("======================================================================================")

In [ ]:
Xte_tb_features

In [ ]:
ytr_tb_features